# Sample Database Code

Steal this code.

The TIdatabase module encapsulates all of the loading, storing and joining of the student, college and applications.

The module should be imported at the beginning of your code, whether it is a regular Python file or an iPython Notebook.

In [6]:
import TIdatabase as ti
import pandas as pd

In [7]:

students = ti.Student()
# populate students with random values
students.fillRandom(10)
students.df 

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,canAfford,GPA,female,MinorityGender,...,outofstate,international,firstinfamily,alumni,sports,artist,program,workexp,schooltype,intendedgradyear
0,T1JCSHM95F,0.449563,0.149376,0.250862,0.203266,0.331007,1,0.998569,-1,-1,...,0,-1,-1,1,1,-1,3,0,1,2014
1,IIGASLK242,0.011935,0.574511,0.109493,0.110674,0.315288,-1,0.615136,-1,1,...,1,1,-1,0,0,0,5,-1,2,2015
2,TKHQ9OC62Y,0.962443,0.141297,0.207493,0.115265,0.584206,0,0.467918,1,-1,...,-1,0,1,1,1,-1,4,-1,2,2013
3,XHXYW2PCE9,0.534247,0.871587,0.641622,0.176225,0.180480,0,0.881910,0,-1,...,0,-1,-1,1,1,1,1,-1,2,2017
4,RKJ1PQOW6D,0.960171,0.692471,0.542503,0.408890,0.103562,-1,0.237146,1,1,...,1,-1,0,1,-1,-1,1,1,2,2012
5,TEN922H47K,0.117865,0.608660,0.058065,0.260044,0.652426,0,0.605323,0,0,...,-1,1,0,0,1,-1,1,0,1,2010
6,JOK9R2RYM2,0.454978,0.983682,0.874345,0.130557,0.082017,0,0.121903,1,0,...,-1,-1,-1,1,-1,-1,5,0,4,2014
7,WRZ382OG27,0.965255,0.600731,0.365645,0.641337,0.572943,-1,0.636701,-1,1,...,-1,1,0,1,-1,-1,2,-1,3,2017
8,AFELNGA8Y7,0.988233,0.491530,0.988586,0.990854,0.220702,1,0.351144,0,1,...,-1,-1,0,0,-1,-1,5,1,5,2011
9,2P3PI9HRLK,0.303543,0.412180,0.774522,0.381915,0.436437,1,0.776652,1,0,...,-1,-1,-1,1,-1,0,5,-1,4,2014


The list of colleges is populated from a CSV stored in the same directory as this notebook. It can be edited using your favorite CSV editor, such as Excel. When you create a new instance of College, the values will be read in from the CSV

In [8]:
# populate with list of known colleges
colleges = ti.College()
colleges.df

,name,acceptrate,size,public,finAidPct,instatePct
0,Harvard,0.05,8000,-1,0.5,0.2
1,Yale,0.06,7000,-1,0.5,0.1
2,Umass,0.80,12000,1,0.8,0.9


This is the table of application forms. "acceptStatus" is our predictor. "acceptProb" is our $\hat{Y}$ probability.

In [9]:
applForm = ti.ApplForm()
applForm.fillRandom(30)
applForm.df

,studentID,collegeID,earlyAppl,visited,acceptStatus,acceptProb
0,IIGASLK242,Yale,-1,0,-1,0
1,TKHQ9OC62Y,Umass,1,0,-1,0
2,AFELNGA8Y7,Harvard,-1,0,1,0
3,RKJ1PQOW6D,Harvard,0,0,0,0
4,T1JCSHM95F,Umass,-1,1,-1,0
5,2P3PI9HRLK,Harvard,1,-1,-1,0
6,WRZ382OG27,Yale,-1,-1,-1,0
7,XHXYW2PCE9,Harvard,1,-1,1,0
8,TEN922H47K,Harvard,0,1,-1,0
9,T1JCSHM95F,Yale,-1,1,1,0


To combine the student and application forms tables, use Pandas merge capability, which will match rows by identical column names, which studentID in this case:

In [13]:
applications = pd.merge(students.df,applForm.df)
applications

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,canAfford,GPA,female,MinorityGender,...,artist,program,workexp,schooltype,intendedgradyear,collegeID,earlyAppl,visited,acceptStatus,acceptProb
0,T1JCSHM95F,0.449563,0.149376,0.250862,0.203266,0.331007,1,0.998569,-1,-1,...,-1,3,0,1,2014,Umass,-1,1,-1,0
1,T1JCSHM95F,0.449563,0.149376,0.250862,0.203266,0.331007,1,0.998569,-1,-1,...,-1,3,0,1,2014,Yale,-1,1,1,0
2,T1JCSHM95F,0.449563,0.149376,0.250862,0.203266,0.331007,1,0.998569,-1,-1,...,-1,3,0,1,2014,Harvard,-1,1,1,0
3,IIGASLK242,0.011935,0.574511,0.109493,0.110674,0.315288,-1,0.615136,-1,1,...,0,5,-1,2,2015,Yale,-1,0,-1,0
4,IIGASLK242,0.011935,0.574511,0.109493,0.110674,0.315288,-1,0.615136,-1,1,...,0,5,-1,2,2015,Harvard,0,0,-1,0
5,IIGASLK242,0.011935,0.574511,0.109493,0.110674,0.315288,-1,0.615136,-1,1,...,0,5,-1,2,2015,Umass,0,-1,0,0
6,TKHQ9OC62Y,0.962443,0.141297,0.207493,0.115265,0.584206,0,0.467918,1,-1,...,-1,4,-1,2,2013,Umass,1,0,-1,0
7,TKHQ9OC62Y,0.962443,0.141297,0.207493,0.115265,0.584206,0,0.467918,1,-1,...,-1,4,-1,2,2013,Yale,0,-1,-1,0
8,TKHQ9OC62Y,0.962443,0.141297,0.207493,0.115265,0.584206,0,0.467918,1,-1,...,-1,4,-1,2,2013,Harvard,1,-1,1,0
9,XHXYW2PCE9,0.534247,0.871587,0.641622,0.176225,0.180480,0,0.881910,0,-1,...,1,1,-1,2,2017,Harvard,1,-1,1,0


Now the applications Pandas DataFrame is ready to use.